In [52]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline


In [53]:
 # Load the dataset
    
start_tok = '<S>'
end_tok = '<E>'
unk_tok = '<U>'
    
def load_dataset(file):
    reviews = open(file, 'r').read().splitlines()
    X = []
    Y = []
    for review in reviews:
        label, text = review.strip().split('|||')
        # Start and End tokens are added - in the final set, add space and unknown tokens as well
        X.append([ start_tok ] + text.strip().split(' ') + [ end_tok ])
        Y.append(int(label))
    print(len(X), len(Y))
    return X, Y
        
Xtrw, Ytrw = load_dataset('data/sst-sentiment-text-threeclass/train.txt')
Xdevw, Ydevw = load_dataset('data/sst-sentiment-text-threeclass/dev.txt')
Xtew, Ytew = load_dataset('data/sst-sentiment-text-threeclass/test.txt')


8544 8544
1101 1101
2210 2210


In [48]:
import random
random.seed(42)

words = { word for x in Xtr for word in x }

stoi = {s:i+1 for i,s in enumerate(sorted(list(words)))}
stoi[unk_tok] = 0
itos = {i:s for s,i in stoi.items()}
wordset_size = len(itos)
print(wordset_size)



868033


In [49]:
readable_labels = {
    -1 : 'negative',
     0 : 'neutral',
     1 : 'positive'
}

readable_labels_size = len(readable_labels)

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return torch.eye(num_classes, dtype=torch.float32)[y]

itol = { i:v for i,v in enumerate(readable_labels) }
ltoi = { v:i for i,v in itol.items() }
itoc = { i:to_categorical(l, num_classes=readable_labels_size) for i,l in itol.items()}
def ctoi(t):
    for i,c in itoc.items():
        if torch.equal(c,t):
            return i
    return -1
itol, ltoi, itoc

({0: -1, 1: 0, 2: 1},
 {-1: 0, 0: 1, 1: 2},
 {0: tensor([0., 0., 1.]), 1: tensor([1., 0., 0.]), 2: tensor([0., 1., 0.])})

In [50]:
# build the dataset
block_size = 64

def build_dataset(X, Y):
    
    Xt, Yt = [], []
    
    for x,y in zip(X, Y):
        # padded context of zero tokens with block size expected
        context = [0] * block_size
        for i, w in enumerate(x):
            try:
              ix = stoi[w]
            except KeyError:
              ix = stoi[unk_tok]
            if i > block_size // 2:
              Xt.append(context)
              Yt.append(itoc[ltoi[y]])
            context = context[1:] + [ix] #crop and append
        Xt.append(context)
        Yt.append(itoc[ltoi[y]] if len(set(context)) > 1 else itoc[ltoi[0]])
    Xt = torch.tensor(Xt)
    Yt = torch.stack(Yt)
    print(Xt.shape, Yt.shape)
    return Xt, Yt

 
Xtr, Ytr = build_dataset(Xtrw, Ytrw)      #80%
Xdev, Ydev = build_dataset(Xdevw, Ydevw)  #10%
Xte, Yte = build_dataset(Xtew, Ytew)      #10%


torch.Size([13563, 64]) torch.Size([13563, 3])
torch.Size([1633, 64]) torch.Size([1633, 3])
torch.Size([3381, 64]) torch.Size([3381, 3])


In [51]:
# Pick ten samples in intervals to show the construction of XTr train set and YTr label set
for x,y in zip(Xtr[:20], Ytr[:20]):
    print(' '.join(itos[ix.item()] for ix in x), '--->', readable_labels[itol[ctoi(y)]])

<U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> ---> positive
<U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> ---> positive
<U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> ---> positive
<U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U> <U

In [ ]:
class Linear:
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # kaiming initialization for normalized variance
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# ---------------
class BatchNorm1d:
  def __init__(self, dim, eps=1e-6, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      dims = [d for d in range(0, x.dim() - 1)]
      xmean = x.mean(tuple(dims), keepdim=True) # batch mean
      xvar = x.var(tuple(dims), keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar

    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

# ---------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

# ---------------

class Embedding:
  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))
        
  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out
        
  def parameters(self):
    return [self.weight]

# ---------------
class FlattenConsecutive:
  def __init__(self, n):
    self.n = n
    
  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T//self.n, C*self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out

  def parameters(self):
    return []

# ---------------
class Sequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

# ---------------

class Softmax:
  def __call__(self, x):
    self.out = torch.softmax(x, dim=1)
    return self.out
  def parameters(self):
    return []

# ---------------

In [ ]:
torch.manual_seed(42)
wordset_size

In [ ]:
n_embd = 10
n_hidden = 24

model = Sequential([
    Embedding(wordset_size, n_embd),
    FlattenConsecutive(4), Linear(n_embd * 4, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    FlattenConsecutive(4), Linear(n_hidden * 4, n_hidden // 2, bias=False), BatchNorm1d(n_hidden // 2), Tanh(),
    FlattenConsecutive(4), Linear(n_hidden * 2, readable_labels_size)
])

    
#parameter init
with torch.no_grad():
    model.layers[-1].weight *= 0.1 #last linear layer is reduced so that we make it less confident at the beginning
    
parameters = model.parameters()
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

In [ ]:
# same optimization as last time
max_steps = 120000
batch_size = 32
lossi = []
lossv = []
ud = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  
  # forward pass
  logits = model(Xb)
  loss = F.cross_entropy(logits, Yb) # loss function
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.01 if i > max_steps/2 else 0.1

  for p in parameters:
    p.data += -lr * p.grad
    
  with torch.no_grad():
    ix = torch.randint(0, Xdev.shape[0], (batch_size,))
    Xv, Yv = Xdev[ix], Ydev[ix]
    vlogits = model(Xv)
    l = F.cross_entropy(vlogits, Yv)
    lossv.append(l.log10().item())

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: loss(test): {loss.item():.4f} :loss(val): {l.item():.4f} : lr:{lr:.3f}' )
    
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])



In [ ]:
for layer in model.layers:
    if layer.out != None:
      print(layer.__class__.__name__, ':', tuple(layer.out.shape))
    else:
      print(layer.__class__.__name__, ':', 'Uninitialized')

In [ ]:
lvi = torch.tensor(lossi).view(-1,1000).mean(1)
lvp = torch.tensor(lossv).view(-1,1000).mean(1)


fig, ax = plt.subplots(figsize=(20, 10), layout='constrained')
ax.plot(lvi, label='training-loss')  
ax.plot(lvp, label='validation-loss')  
ax.set_title("Training and Validation Loss") 
ax.legend();  # Add a legend.

In [ ]:
for layer in model.layers:
    print(layer.__class__.__name__, ':', tuple(layer.out.shape))
    

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(model.layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out.grad
    if t != None:
      print('layer %d (%10s): mean %+f, std %e' % (i, layer.__class__.__name__, t.mean(), t.std()))
      hy, hx = torch.histogram(t, density=True)
      plt.plot(hx[:-1].detach(), hy.detach())
      legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('gradient distribution')


In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(model.layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh) or isinstance(layer, Softmax):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' % (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('activation distribution')


In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i,p in enumerate(parameters):
  t = p.grad
  if p.ndim == 2:
    print('weight %10s | mean %+f | std %e | grad:data ratio %e' % (tuple(p.shape), t.mean(), t.std(), t.std() / p.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'{i} {tuple(p.shape)}')
plt.legend(legends)
plt.title('weights gradient distribution');

In [ ]:
plt.figure(figsize=(20, 4))
legends = []
for i,p in enumerate(parameters):
  if p.ndim == 2:
    plt.plot([ud[j][i] for j in range(len(ud))])
    legends.append('param %d' % i)
plt.plot([0, len(ud)], [-3, -3], 'k') # these ratios should be ~1e-3, indicate on plot
plt.legend(legends);

In [ ]:
for layer in model.layers:
    layer.training = False
    


In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]

  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')
split_loss('test')

In [ ]:

@torch.no_grad()
def test_dataset(split):
  X,Y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]  

  correct = 0
  total = 0
  trace = 5
  for i in range(X.shape[0] // batch_size):
    ix = torch.arange(i*batch_size, (i+1)*batch_size)
    Xc, Yc = X[ix], Y[ix] # batch X,Y

    logits = model(Xc)
    probs = F.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1)

    target = []
    for t in preds.tolist():
      target.append(to_categorical(t, num_classes=readable_labels_size))
    target = torch.stack(target)

    for x,y,p,t in zip(Xc, Yc, probs, target):
      act = readable_labels[itol[ctoi(y)]]
      pred = readable_labels[itol[ctoi(t)]]
      if act == pred:
        correct += 1
      total += 1
      if total % 1000 == 0:
        trace = 5
      if trace > 0:
        print(' '.join(itos[ix.item()] for ix in x), '\n\t---> Actual:', act, '---> Predicted:', pred, f' probabaility : {torch.max(p).item()*100:.2f}%',  )
        trace -= 1
  return correct, total

correct,total = test_dataset('train')
print(f'[TRAIN] Correct Sentiment: {correct} out of {total}, Accuracy : {correct/total * 100:.2f}%')

correct,total = test_dataset('val')
print(f'[VAL] Correct Sentiment: {correct} out of {total}, Accuracy : {correct/total * 100:.2f}%')

correct,total = test_dataset('test')
print(f'[TEST] Correct Sentiment: {correct} out of {total}, Accuracy : {correct/total * 100:.2f}%')
